### 날씨 데이터 파싱
* [기상청날씨데이터](http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp)
* 파싱한 데이터를 dict, list 자료구조에 저장
* 자료구조의 데이터를 json 파일로 저장
* BeautifulSoup 의 find(), find_all(), select(), select_one() 함수 비교

In [1]:
!pip3 show lxml

Name: lxml
Version: 5.2.1
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD-3-Clause
Location: C:\Users\r2com\anaconda3\Lib\site-packages
Requires: 
Required-by: parsel, Scrapy


In [2]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, features='xml')
    
    title_tag = soup.find('title')
    print(type(title_tag), title_tag, title_tag.text)
    
    title_all_tag = soup.find_all('title')
    print(type(title_all_tag), title_all_tag)
    
    location_tag = soup.find('location', attrs={'wl_ver':3})
    print('==find() ', type(location_tag))

    location_tag2 = soup.select_one("location[wl_ver='3']")
    print('==select_one() ',type(location_tag2))

    loc_tag_all = soup.findAll('location', attrs={'wl_ver':3})
    print('==find_all() ', type(loc_tag_all), len(loc_tag_all))

    loc_tag_all2 = soup.select("location[wl_ver='3']")
    print('==select() ', type(loc_tag_all2))

200
<class 'bs4.element.Tag'> <title>기상청 육상 중기예보</title> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2024년 08월 06일 (화)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
==find()  <class 'bs4.element.Tag'>
==select_one()  <class 'bs4.element.Tag'>
==find_all()  <class 'bs4.element.ResultSet'> 41
==select()  <class 'bs4.element.ResultSet'>


'''
<location wl_ver="3">
    <province>서울ㆍ인천ㆍ경기도</province>
    <city>서울</city>
    <data>
        <mode>A02</mode>
        <tmEf>2022-08-28 00:00</tmEf>
        <wf>맑음</wf>
        <tmn>18</tmn>
        <tmx>28</tmx>
        <reliability/>
        <rnSt>0</rnSt>
    </data>
    <data>
        <mode>A02</mode>
        <tmEf>2022-08-28 12:00</tmEf>
        <wf>구름많음</wf>
        <tmn>18</tmn>
        <tmx>28</tmx>
        <reliability/>
        <rnSt>30</rnSt>
    </data>
</location>
{
  "province":"서울ㆍ인천ㆍ경기도",
  "city":"서울",
  "datas":[{"mode:"A02","tmEf":"2022-01-31 00:00","wf":"맑음"},
            {"mode:"A02”,"tmEf":"2022-01-31 12:00","wf":"구름많음"},{},{}]
}
'''

### 서울(city)의 날씨 데이터 Parsing

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
if res.ok:
    soup = BeautifulSoup(res.text, features='xml')
    location_dict = {} #dict()
    location_tag = soup.find('location',attrs={'wl_ver':3})
    print(location_tag)

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0">
<channel>
<title>기상청 육상 중기예보</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</link>
<description>기상청 날씨 웹서비스</description>
<language>ko</language>
<generator>기상청</generator>
<pubDate>2024년 08월 06일 (화)요일 06:00</pubDate>
<item>
<author>기상청</author>
<category>육상중기예보</category>
<title>전국 육상 중기예보 - 2024년 08월 06일 (화)요일 06:00 발표</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</link>
<guid>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</guid>
<description>
<header>
<title>전국 육상중기예보</title>
<tm>202408060600</tm>
<wf>○ (강수) 9일(금) 오후에는 경상권에 소나기가 오겠습니다.&lt;br /&gt;○ (기온) 이번 예보기간 아침 기온은 24~27도, 낮 기온은 30~35도로 평년(최저기온 22~24도, 최고기온 29~32도)보다 높겠습니다.&lt;br /&gt;○ (주말전망) 10일(토)~11일(일)은 전국이 구름많겠습니다. 아침 기온은 24~27도, 낮 기온은 31~35도가 되겠습니다.&lt;br /&gt;&lt;br /&gt;* 이번 예보기간 동안 최고체감온도가 35도 내외로 올라 무더위와 열대야가 나타나는 날이 많겠고, 대기 불안정으로 인한 소나기가 내릴 가능성이 있겠으니, 앞으로 발표되는 최신 예보를 참고하기 바랍니다.</wf>
</header>
<body>
<location w

### 41개 City의  날씨 데이터 파싱

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
if res.ok:
    pass

In [6]:
import warnings
warnings.filterwarnings('ignore')

#### weather.json 파일로 저장하기

#### weather.json 파일을 읽어오기